**Copyright 2020 The TensorFlow Authors.**

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Clustering de pesos no exemplo do Keras

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/clustering/clustering_example"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/model_optimization/guide/clustering/clustering_example.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/model_optimization/guide/clustering/clustering_example.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/model_optimization/guide/clustering/clustering_example.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

Bem-vindo ao exemplo completo de *clustering de peso*, parte do Kit de ferramentas para otimização de modelos do TensorFlow.

### Outras páginas

Para uma introdução sobre o que é o clustering de peso e para determinar se você deve usá-lo (incluindo a compatibilidade dele), veja a [página de visão geral](https://www.tensorflow.org/model_optimization/guide/clustering).

Para encontrar rapidamente as APIs necessárias para seu caso de uso (além de realizar todo o clustering de um modelo com 16 clusters), veja o [guia completo](https://www.tensorflow.org/model_optimization/guide/clustering/clustering_comprehensive_guide).

### Conteúdo

Neste tutorial, você:

1. Treinará um modelo `tf.keras` para o dataset MNIST do zero.
2. Ajustará o modelo ao aplicar a API de clustering de peso e verá a exatidão.
3. Criará modelos do TF e do TFLite 6x menores com o clustering.
4. Criará um modelo do TFLite 8x menor ao combinar o clustering de peso e a quantização pós-treinamento.
5. Verá a persistência da exatidão do TF para o TFLite.

## Configuração

Você pode executar este Notebook do Jupyter no seu [virtualenv](https://www.tensorflow.org/install/pip?lang=python3#2.-create-a-virtual-environment-recommended) local ou no [colab](https://colab.sandbox.google.com/). Para mais detalhes sobre como configurar as dependências, consulte o [guia de instalação](https://www.tensorflow.org/model_optimization/guide/install). 

In [ ]:
! pip install -q tensorflow-model-optimization

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import tempfile
import zipfile
import os

## Treine um modelo tf.keras para o MNIST sem o clustering

In [ ]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images  = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation=tf.nn.relu),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_images,
    train_labels,
    validation_split=0.1,
    epochs=10
)

### Avalie o modelo de linha de base e salve-o para usar mais tarde

In [ ]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

## Ajuste o modelo pré-treinado com o clustering

Aplique a API `cluster_weights()` a um modelo pré-treinado inteiro para demonstrar a eficácia em reduzir o tamanho do modelo após a aplicação do zip, mantendo uma exatidão adequada. Para saber como melhor equilibrar a exatidão e a taxa de compressão no seu caso de uso, consulte o exemplo por camada no [guia completo](https://www.tensorflow.org/model_optimization/guide/clustering/clustering_comprehensive_guide).


### Defina o modelo e aplique a API de clustering

Antes de passar o modelo à API de clustering, confira se ele foi treinado e apresenta uma exatidão aceitável.

In [ ]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 16,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}

# Cluster a whole model
clustered_model = cluster_weights(model, **clustering_params)

# Use smaller learning rate for fine-tuning clustered model
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

clustered_model.summary()

### Ajuste o modelo e compare a exatidão dele com a linha de base

Ajuste o modelo com o clustering para 1 época.

In [ ]:
# Fine-tune model
clustered_model.fit(
  train_images,
  train_labels,
  batch_size=500,
  epochs=1,
  validation_split=0.1)

Nesse exemplo, há uma perda mínima na exatidão do teste após o clustering, em comparação com a linha de base.

In [ ]:
_, clustered_model_accuracy = clustered_model.evaluate(
  test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Clustered test accuracy:', clustered_model_accuracy)

## Crie modelos **6x** menores com o clustering

O `strip_clustering` e a aplicação de um algoritmo de compressão padrão (por exemplo, por gzip) são necessários para ver os benefícios de compressão do clustering.

Primeiro, crie um modelo comprimível para o TensorFlow. Aqui, o `strip_clustering` remove todas as variáveis (por exemplo, `tf.Variable` para armazenar os centroides dos clusters e os índices) que só são necessárias para o clustering durante o treinamento e aumentariam o tamanho do modelo durante a inferência.

In [ ]:
final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

_, clustered_keras_file = tempfile.mkstemp('.h5')
print('Saving clustered model to: ', clustered_keras_file)
tf.keras.models.save_model(final_model, clustered_keras_file, 
                           include_optimizer=False)

Em seguida, crie modelos comprimíveis para o TFLite. Você pode converter o modelo agrupado em um formato executável no seu back-end de destino. O TensorFlow Lite é um exemplo que você pode usar para a implantação em dispositivos móveis.

In [ ]:
clustered_tflite_file = '/tmp/clustered_mnist.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
tflite_clustered_model = converter.convert()
with open(clustered_tflite_file, 'wb') as f:
  f.write(tflite_clustered_model)
print('Saved clustered TFLite model to:', clustered_tflite_file)

Defina uma função helper para comprimir os modelos por gzip e medir o tamanho compactado.

In [ ]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

Compare e veja se os modelos estão **6x** menores com o clustering.

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered Keras model: %.2f bytes" % (get_gzipped_model_size(clustered_keras_file)))
print("Size of gzipped clustered TFlite model: %.2f bytes" % (get_gzipped_model_size(clustered_tflite_file)))

## Crie um modelo do TFLite **8x** menor ao combinar o clustering de peso e a quantização pós-treinamento

Você pode aplicar a quantização pós-treinamento ao modelo agrupado para benefícios adicionais.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

_, quantized_and_clustered_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_clustered_tflite_file, 'wb') as f:
  f.write(tflite_quant_model)

print('Saved quantized and clustered TFLite model to:', quantized_and_clustered_tflite_file)
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_clustered_tflite_file)))

## Veja a persistência da exatidão do TF para o TFLite

Defina uma função helper para avaliar o modelo do TFLite com o dataset de teste.

In [ ]:
def eval_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

Avalie o modelo após o clustering e a quantização e veja se a exatidão do TensorFlow persiste no back-end do TFLite.

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_quant_model)
interpreter.allocate_tensors()

test_accuracy = eval_model(interpreter)

print('Clustered and quantized TFLite test_accuracy:', test_accuracy)
print('Clustered TF test accuracy:', clustered_model_accuracy)

## Conclusão

Neste tutorial, você viu como criar modelos agrupados com a API do Kit de ferramentas para otimização de modelos do TensorFlow. Mais especificamente, você conferiu um exemplo completo para criar um modelo 8x menor para o MNIST com uma diferença mínima de exatidão. Recomendamos que você teste essa nova capacidade, que pode ser especialmente importante para a implantação em ambientes de recursos limitados.
